# Notebook 5: Composing Examples

In this notebook we will examine how we can use the elements we have encountered so far, in order to construct a examples which will allow us to train machine learning models with data generated in real time. This is a core functionality of GravyFlow.

As usual, we begin by performing the relevent imports:

In [16]:
# Built-in imports
from typing import List, Dict, Iterator
from pathlib import Path

# Dependency imports: 
import numpy as np
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module.
import gravyflow as gf

## Creating an example generator through composition

We can combine all the elements we have seen so, noise generation, waveform generation, and waveform projection, and use MLy to create a custom Python generator.

In order to scale the injection with respect to the noise we can use a `gf.ScalingMethod` object. GravyFlow supports scaling with SNR (`gf.ScalingTypes.SNR`), HRSS (`gf.ScalingTypes.HRSS`), and HPEAK (`gf.ScalingTypes.HPEAK`).

- `value` : `Union[gf.Distribution, np.ndarray]`, Required
  > The value or distribution to use to scale the injections, units vary depending on type parameter.

- `type_` : `gf.ScalingTypes`, Required
  > Type of scaling, one of either `gf.ScalingTypes.SNR`, `gf.ScalingTypes.HRSS`, or `gf.ScalingTypes.HPEAK`.

Let us create a `gf.ScalingMethod` object:

In [17]:
# Create a scaling method object in order to scale the injection to the noise:
scaling_method : gf.ScalingMethod = gf.ScalingMethod(
    value=gf.Distribution(
        value=20,
        type_=gf.DistributionType.CONSTANT
    ),
    type_=gf.ScalingTypes.SNR
)

Next, we can set up all the other elements that we can use to compose or example generator:

In [18]:
with gf.env():
    # This object will be used to obtain real interferometer data based on specified parameters.
    ifo_data_obtainer : gf.IFODataObtainer = gf.IFODataObtainer(
        observing_runs=gf.ObservingRun.O3, # Specify the observing run (e.g., O3).
        data_quality=gf.DataQuality.BEST,  # Choose the quality of the data (e.g., BEST).
        data_labels=[                      # Define the types of data to include.
            gf.DataLabel.NOISE, 
            gf.DataLabel.GLITCHES
        ],
        segment_order=gf.SegmentOrder.RANDOM, # Order of segment retrieval (e.g., RANDOM).
        force_acquisition=True,               # Force the acquisition of new data.
        cache_segments=False                  # Choose not to cache the segments.
    )

    # Initialize the noise generator wrapper:
    # This wrapper will use the ifo_data_obtainer to generate real noise based on the specified parameters.
    noise: gf.NoiseObtainer = gf.NoiseObtainer(
        ifo_data_obtainer=ifo_data_obtainer, # Use the previously set up IFODataObtainer object.
        noise_type=gf.NoiseType.REAL,        # Specify the type of noise as REAL.
        ifos=gf.IFO.L1 # Specify the interferometer (e.g., LIGO Livingston L1).
    )

    # Define a uniform distribution for the mass of the first object in solar masses.
    mass_1_distribution_msun : gf.Distribution = gf.Distribution(
        min_=10.0, 
        max_=60.0, 
        type_=gf.DistributionType.UNIFORM
    )

    # Define a uniform distribution for the mass of the second object in solar masses.
    mass_2_distribution_msun : gf.Distribution = gf.Distribution(
        min_=10.0, 
        max_=60.0, 
        type_=gf.DistributionType.UNIFORM
    )

    # Define a uniform distribution for the inclination of the binary system in radians.
    inclination_distribution_radians : gf.Distribution = gf.Distribution(
        min_=0.0, 
        max_=np.pi, 
        type_=gf.DistributionType.UNIFORM
    )

    # Initialize a PhenomD waveform generator with the defined distributions.
    # This generator will produce waveforms with randomly varied masses and inclination angles.
    phenom_d_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
        mass_1_msun=mass_1_distribution_msun,
        mass_2_msun=mass_2_distribution_msun,
        inclination_radians=inclination_distribution_radians,
        scaling_method=scaling_method,
        injection_chance=0.5 # Set so half produced examples will not contain this signal
    )
    
    generator : Iterator = gf.data(       
        noise_obtainer=noise,
        waveform_generators=phenom_d_generator,
        num_examples_per_batch=8,
        input_variables=[
            gf.ReturnVariables.WHITENED_ONSOURCE, 
        ],
        output_variables=[
            gf.ReturnVariables.INJECTIONS, 
            gf.ReturnVariables.WHITENED_INJECTIONS,
            gf.ReturnVariables.INJECTION_MASKS
        ]
    )

2025-02-11 10:56:54,533 - INFO - Available GPUs: ['5']
2025-02-11 10:56:54,535 - INFO - Visible GPUs after restriction: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]
2025-02-11 10:56:54,569 - INFO - Recomputing valid segments!


5


2025-02-11 10:56:55,046 - INFO - Loading event times from cache.


Similiarly to the individual elements, we can use this example generator as an iterator, and produce N examples for use to examine:

In [19]:
with gf.env():
    # Generate a batch of examples using the composed generator.
    input_data, output_data = next(generator)

2025-02-11 10:56:56,196 - INFO - Available GPUs: ['5']
2025-02-11 10:56:56,200 - INFO - Visible GPUs after restriction: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]


5


We can print the output of the generator to examine its format, which values are returned will depend on which parameters we have requested in the input_variables, and output_variables field in our gf.data function. Both are returned in the form of a dictionary, which can easly be fed into a keras model if the inputs of the model are named similarly to the variable feilds, we will show an example of this later in the notebook.

In [20]:
# This is the data we will uses as an input examples to out model:
print(f"Dictionary to feed the model: \n {input_data}")

# This is the target data we will use as labels to train our model:
print(f"Dictionary to use as the model labels: \n {output_data}")

Dictionary to feed the model: 
 {'WHITENED_ONSOURCE': <tf.Tensor: shape=(8, 1, 2048), dtype=float16, numpy=
array([[[ 1.84  , -0.576 , -0.5933, ..., -0.3196, -0.3562,
         -0.7817]],

       [[ 1.042 ,  1.724 ,  0.738 , ..., -0.456 , -1.083 ,
          1.209 ]],

       [[ 1.011 ,  0.859 , -0.1366, ...,  1.289 ,  0.6846,
          0.2737]],

       ...,

       [[-0.1262, -0.49  , -0.6177, ...,  0.7026, -2.033 ,
         -0.8545]],

       [[ 0.6216, -1.045 , -1.036 , ..., -0.4414,  1.321 ,
         -0.8643]],

       [[ 0.5684,  1.71  , -1.056 , ...,  1.073 , -0.955 ,
         -0.3467]]], dtype=float16)>}
Dictionary to use as the model labels: 
 {'INJECTIONS': <tf.Tensor: shape=(8, 1, 2048), dtype=float32, numpy=
array([[[ 0.21530649,  0.22814944,  0.23822331, ...,
          0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,
          0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,
          0

We can then print some examples from this dataset to examine the output:

In [21]:
# Extract the data from the generator output: 
onsource: tf.Tensor = input_data[gf.ReturnVariables.WHITENED_ONSOURCE.name]
injections: tf.Tensor = output_data[gf.ReturnVariables.INJECTIONS.name][0]
whitened_injections: tf.Tensor = output_data[gf.ReturnVariables.WHITENED_INJECTIONS.name][0]
injection_masks: tf.Tensor = output_data[gf.ReturnVariables.INJECTION_MASKS.name][0]

print(onsource)
print(injections)

# Initialize an empty layout for the strain plots.
generated_data_layout: List = []

# Iterate over the multi-injections and their corresponding parameters.
for onsource_, injection, whitened_injection, masks_ in zip(
        onsource,
        injections,
        whitened_injections,
        injection_masks
    ):
    # Create strain plots for each Phenom D and WNB injection with titles displaying the parameter values.
    generated_data_layout.append([
        gf.generate_strain_plot(
            {
                "Onsouce": onsource_,
                "Whitened Injection": whitened_injection,
                "Injection": injection,
            },
            height=400,
            width=800,
            title=f"Example Output. Has Injection: {bool(masks_)}"
        )
    ])

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(generated_data_layout)
output_notebook()
show(grid)

tf.Tensor(
[[[ 1.84   -0.576  -0.5933 ... -0.3196 -0.3562 -0.7817]]

 [[ 1.042   1.724   0.738  ... -0.456  -1.083   1.209 ]]

 [[ 1.011   0.859  -0.1366 ...  1.289   0.6846  0.2737]]

 ...

 [[-0.1262 -0.49   -0.6177 ...  0.7026 -2.033  -0.8545]]

 [[ 0.6216 -1.045  -1.036  ... -0.4414  1.321  -0.8643]]

 [[ 0.5684  1.71   -1.056  ...  1.073  -0.955  -0.3467]]], shape=(8, 1, 2048), dtype=float16)
tf.Tensor(
[[0.21530649 0.22814944 0.23822331 ... 0.         0.
  0.        ]], shape=(1, 2048), dtype=float32)


ValueError: The value for key 'Injection' is not an np.ndarray.